In [20]:
import openeo
import geopandas as gpd
import leafmap
import shapely 

from utils_gapfilling import calculate_snow



Features to use for training the model:
- DEM, slope and aspect
- NDSI
- date (doy)
- global snow cover

Selezionare punti random (1000 punti) e prendere su tutte le feature

In [2]:
eoconn = openeo.connect('https://openeo.dataspace.copernicus.eu/', auto_validate=False)
eoconn.authenticate_oidc()

eoconn.describe_account()

Authenticated using refresh token.


{'info': {'oidc_userinfo': {'email': 'valentina.premier@eurac.edu',
   'email_verified': True,
   'family_name': 'Premier',
   'given_name': 'Valentina',
   'name': 'Valentina Premier',
   'preferred_username': 'valentina.premier@eurac.edu',
   'sub': '1aa67f75-e178-495e-a131-9f7dffaaae18'}},
 'name': 'Valentina Premier',
 'user_id': '1aa67f75-e178-495e-a131-9f7dffaaae18'}

In [9]:
shp_path = r'/mnt/CEPH_PROJECTS/PROSNOW/research_activity/Senales/auxiliary/boundaries/SenalesCatchment/SenalesCatchment.shp'

catchment_outline = gpd.read_file(shp_path)
bbox = catchment_outline.bounds.iloc[0]

center = (float(catchment_outline.centroid.y), 
          float(catchment_outline.centroid.x))
m = leafmap.Map(center=center, zoom=1)
m.add_vector(shp_path, layer_name="catchment")
m

Map(center=[46.72634837079619, 10.877755221904536], controls=(ZoomControl(options=['position', 'zoom_in_text',…

In [10]:
eps= 0.01 #0.05 # buffer of ca. 500 m
startdate = '2023-08-02'
enddate = '2024-08-15'
cloud_prob = 100

scl = eoconn.load_collection(
    "SENTINEL2_L2A",
    temporal_extent=[startdate, enddate],
    spatial_extent={'west':bbox[0]-eps,
                    'east':bbox[2]+eps,
                    'south':bbox[1]-eps,
                    'north':bbox[3]+eps,
                    'crs':4326},   
    bands=['SCL'],
    max_cloud_cover=cloud_prob,
)

snow = calculate_snow(scl)

In [37]:
# adjust automatically this part
west=631810.
south=5167710.
east=655790.
north=5184190.
area = shapely.geometry.box(west, south, east, north) 
res = 20.

# check that the grid is aligned with a 500 m gri
print('Nr. of low-resolution pixels in the resampled grid (x): {}'.format((east + res/2. - (west - res/2. ))/500.))
print('Nr. of low-resolution pixels in the resampled grid (y): {}'.format((south - res/2. - (north + res/2. ))/-500.))

snow_rsmpl = snow.resample_spatial(resolution=res, 
                                 projection=32632,
                                 method = "nearest")

            
snow_bbox = snow_rsmpl.filter_bbox(west=west, 
                                 south=south, 
                                 east=east, 
                                 north=north, 
                                 crs=32632)
snow_bbox
#snow_bbox.download('snow_bbox.nc')

Nr. of low-resolution pixels in the resampled grid (x): 48.0
Nr. of low-resolution pixels in the resampled grid (y): 33.0


Load DEM

In [26]:
dem_datacube = eoconn.load_collection(
    "COPERNICUS_30",
    spatial_extent={'west':bbox[0]-eps,
                    'east':bbox[2]+eps,
                    'south':bbox[1]-eps,
                    'north':bbox[3]+eps,
                    'crs':4326}, 
    bands=["DEM"]
)


In [27]:
dem_datacube = dem_datacube.drop_dimension("bands")
dem_datacube = dem_datacube.resample_cube_spatial(snow_bbox, method="bilinear")
dem_datacube.download('dem_senales.tif')

In [28]:
# Load the UDF from a file.
udf_path="/home/vpremier/OEMC/SCA/udf-example/udf-slope.py"

slope_func = openeo.UDF.from_file(udf_path, context={"from_parameter": "context"})


# Apply the UDF to the data cube.
slope = dem_datacube.apply_neighborhood(
    slope_func,
    size=[
        {"dimension": "x", "value": 224, "unit": "px"},
        {"dimension": "y", "value": 224, "unit": "px"},
    ],
    overlap=[
        {"dimension": "x", "value": 16, "unit": "px"},
        {"dimension": "y", "value": 16, "unit": "px"}
    ],
)
#slope.download('slope.tif')

In [30]:
# Load the UDF from a file.
udf_path="/home/vpremier/OEMC/SCA/udf-example/udf-aspect.py"

aspect_func = openeo.UDF.from_file(udf_path, context={"from_parameter": "context"})


# Apply the UDF to the data cube.
aspect = dem_datacube.apply_neighborhood(
    aspect_func,
    size=[
        {"dimension": "x", "value": 224, "unit": "px"},
        {"dimension": "y", "value": 224, "unit": "px"},
    ],
    overlap=[
        {"dimension": "x", "value": 16, "unit": "px"},
        {"dimension": "y", "value": 16, "unit": "px"}
    ],
)
#aspect.download('aspect.tif')

In [ ]:
# Get the DOY
t = snow_bbox.dimension_labels('t')

In [35]:
import numpy as np
import geopandas as gpd
from shapely.geometry import Point

n = 20

# Generate random coordinates
y_vect = np.random.uniform(north, south, n)
x_vect = np.random.uniform(east, west, n)

# Create points from these coordinates
points = [Point(x, y) for x, y in zip(x_vect, y_vect)]

# Create a GeoDataFrame
gdf = gpd.GeoDataFrame(geometry=points)

# Set the coordinate reference system (CRS) to WGS84 (EPSG:4326)
gdf.set_crs(epsg=32632, inplace=True)

# Save to a Geopackage file
gdf.to_file("random_points.gpkg", layer='random_points', driver="GPKG")


In [33]:
points

[<POINT (646610.162 5171471.118)>,
 <POINT (655073.771 5181195.017)>,
 <POINT (652683.34 5172289.481)>,
 <POINT (653276.843 5169875.257)>,
 <POINT (652589.704 5171088.845)>,
 <POINT (647889.46 5173375.619)>,
 <POINT (635031.758 5183493.377)>,
 <POINT (647483.018 5178818.422)>,
 <POINT (642130.526 5171219.798)>,
 <POINT (638961.77 5175337.116)>,
 <POINT (639758.186 5177336.19)>,
 <POINT (655003.776 5175749.61)>,
 <POINT (639171.286 5180219.947)>,
 <POINT (653249.892 5176379.576)>,
 <POINT (652079.062 5181496.897)>,
 <POINT (639935.44 5175559.643)>,
 <POINT (643004.814 5183343.485)>,
 <POINT (651236.055 5175442.126)>,
 <POINT (632757.15 5170211.367)>,
 <POINT (634455.714 5180333.79)>]

In [45]:
import openeo
from openeo.api.process import Parameter
import logging

from scipy.signal.windows import gaussian
import numpy as np
from typing import List, Optional
_log = logging.getLogger(__name__)


def day_of_month_calc(input: openeo.DataCube) -> openeo.DataCube:
    """
    Calculate day of month.
    Args:
        input (DataCube): Input data cube.
    Returns:
        DataCube: Day of month data cube.
    """ 
    label = Parameter('label')
    day = lambda x:15 + x.process("date_difference",
                                date1=x.process("date_replace_component",date=label,value=15,component="day"),
                                date2=label,unit="day") 
    return input.array_apply(day) 

def calculate_date_score(scl: openeo.DataCube) -> openeo.DataCube:
    """
    Calculate date score from SCL data.
    Args:
        scl (DataCube): SCL data cube.
    Returns:
        DataCube: Date score data cube.
    """
    _log.info(f'calculating date score')
    # Calculate date score
    day_of_month = scl.apply_neighborhood(
        day_of_month_calc,
        size=[{'dimension': 'x', 'unit': 'px', 'value': 1},
              {'dimension': 'y', 'unit': 'px', 'value': 1},
              {'dimension': 't', 'value': "month"}],
        overlap=[]
    )
    return day_of_month


In [46]:
date_score = calculate_date_score(scl)

In [47]:
d = date_score.execute()

ReadTimeout: HTTPSConnectionPool(host='openeo.dataspace.copernicus.eu', port=443): Read timed out. (read timeout=1800)

In [ ]:
d